# DataFrames.jl 介紹 (二): Joins 與 Split-Apply-Combine Strategy

## Day 018 作業1：The Split-Apply-Combine Strategy

同 Day 017，請載入 2020/4/2 COVID-19 Daily Report 資料集，計算每個國家/地區的累計確診人數，並列出所有資料筆數。

In [1]:
using DataFrames, CSV

In [2]:
df = CSV.read("04-02-2020.csv")

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat
,Int64⍰,String⍰,String⍰,String,String,Float64⍰
1,45001,Abbeville,South Carolina,US,2020-04-02 23:25:27,34.2233
2,22001,Acadia,Louisiana,US,2020-04-02 23:25:27,30.2951
3,51001,Accomack,Virginia,US,2020-04-02 23:25:27,37.7671
4,16001,Ada,Idaho,US,2020-04-02 23:25:27,43.4527
5,19001,Adair,Iowa,US,2020-04-02 23:25:27,41.3308
6,29001,Adair,Missouri,US,2020-04-02 23:25:27,40.1906
7,40001,Adair,Oklahoma,US,2020-04-02 23:25:27,35.8849
8,8001,Adams,Colorado,US,2020-04-02 23:25:27,39.8743
9,16003,Adams,Idaho,US,2020-04-02 23:25:27,44.8933


計算計算每個國家/地區的累計確診人數

【提示】可使用 groupby 相關的函式

In [8]:
# groupby 地區, 算 length
by(df, :Province_State, count = :Province_State => length)

,Province_State,count
,String⍰,Int64
1,South Carolina,46
2,Louisiana,61
3,Virginia,103
4,Idaho,30
5,Iowa,64
6,Missouri,74
7,Oklahoma,54
8,Colorado,52
9,Illinois,62


In [6]:
# 顯示所有資料
# groupby 國家, 算 length
by(df, :Country_Region, count = :Country_Region => length)

,Country_Region,count
,String,Int64
1,US,2312
2,Canada,15
3,United Kingdom,10
4,China,33
5,Netherlands,5
6,Australia,8
7,Denmark,3
8,France,10
9,Afghanistan,1


## 作業2：請列出美國各區域或事件 (依 Province_State 欄) 的累積確診、死亡、康復人數。

【提示】使用 Split-Apply-Combine Strategy 時，有幾種不同的函式可以使用，請參照 `DataFrames.jl` 官方文件 [https://juliadata.github.io/DataFrames.jl/stable/lib/functions/#DataFrames.groupby](https://juliadata.github.io/DataFrames.jl/stable/lib/functions/#DataFrames.groupby)

In [13]:
# 輸出df欄位名稱

names(df)

12-element Array{Symbol,1}:
 :FIPS          
 :Admin2        
 :Province_State
 :Country_Region
 :Last_Update   
 :Lat           
 :Long_         
 :Confirmed     
 :Deaths        
 :Recovered     
 :Active        
 :Combined_Key  

In [15]:
# 美國各區域, 累積確診、死亡、康復同時顯示

by(df, :Country_Region, [:Country_Region => length, :Deaths => length, :Recovered => length])

,Country_Region,Country_Region_length,Deaths_length,Recovered_length
,String,Int64,Int64,Int64
1,US,2312,2312,2312
2,Canada,15,15,15
3,United Kingdom,10,10,10
4,China,33,33,33
5,Netherlands,5,5,5
6,Australia,8,8,8
7,Denmark,3,3,3
8,France,10,10,10
9,Afghanistan,1,1,1
